In [ ]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import scipy
%matplotlib inline

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
img_height = 224
img_width = 224
batchsize  = 32

test_datagen = ImageDataGenerator(rescale=1./255)               # no data augmentation performed, only scaling from 0 to 1

train_generator = test_datagen.flow_from_directory(
         r'/notebooks/Kaggle_FinalData_Preprocessed_V3/train',
        target_size=(img_height, img_width),
        batch_size=batchsize,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
         r'/notebooks/Kaggle_FinalData_Preprocessed_V3/validation',
        target_size=(img_height, img_width),
        batch_size=batchsize,
        class_mode='categorical')

In [ ]:
train_img, labels = next(train_generator)

In [ ]:
def plotImage(images_arr):
    fig, axes = plt.subplots(2, 5, figsize=(16,8))
    axes = axes.flatten()
    for img, img_y, ax in zip(images_arr, labels, axes):
        ax.imshow(img)
        ax.set_title('Severity {}'.format(np.argmax(img_y, -1)))
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImage(train_img)
#print(labels)  # uncomment this to check if the severity is correct.
print(train_img.shape[1:])

## Building the VGG19 Model

In [ ]:
from keras.models import Model
from keras.applications import vgg19
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Input, GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda

In [ ]:
input_shape = (224, 224, 3)
vgg = vgg19.VGG19(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)
num_class = 5

In [ ]:
import pandas as pd
vgg.trainable = True
set_trainable = False
for layer in vgg.layers:
    if layer.name in ['block5_conv1', 'block4_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
layers = [(layer, layer.name, layer.trainable) for layer in vgg.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])   

In [ ]:
t_x, t_y = next(train_generator)

in_lay = Input(t_x.shape[1:])

pt_features = vgg(in_lay)

In [ ]:
dropout1 = Dropout(0.6)(pt_features)
x = Flatten()(dropout1)
dense1 = Dense(512, activation='relu')(x)
dense2 = Dense(32, activation='relu')(dense1)
prediction = Dense(num_class, activation='softmax')(dense2)

In [ ]:
model = Model(inputs=in_lay, outputs=prediction)
print(model.summary())

## Adding more performance metrics

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Built In Tensorflow Metrics

In [ ]:
import tensorflow_addons as tfa
from tensorflow_addons.metrics import CohenKappa
from keras.metrics import AUC, TopKCategoricalAccuracy, SpecificityAtSensitivity, Precision, Recall
cohen = tfa.metrics.CohenKappa(num_classes=5)
cohen_linear = tfa.metrics.CohenKappa(num_classes=5, weightage='linear')
cohen_quad = tfa.metrics.CohenKappa(num_classes=5, weightage='quadratic')

In [ ]:
from keras.optimizers import SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam
epochs = 50
lrate = 0.001
opt = SGD(lr=lrate)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy','categorical_accuracy',f1_m,recall_m,precision_m,
                       SpecificityAtSensitivity(0.95), AUC(),cohen_quad])

## Keras Callbacks

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_Weights.hdf5".format('VGG19_PSO_V1_test')
filepath = r'Saved Weights/VGG19/weight_path'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=2, 
                             save_best_only=True, mode='min', save_weights_only = True)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=15, verbose=1, mode='auto')
# Reducing the Learning Rate if result is not improving. 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto',
                              verbose=1)
callbacks_list = [early_stop, reduce_lr]

In [ ]:
from keras.optimizers import SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam

def cnn_for_pso(opt):
    input_shape = (224, 224, 3)
    vgg = vgg19.VGG19(include_top=False, weights='imagenet', 
                                         input_shape=input_shape)
    num_class = 5

    import pandas as pd
    vgg.trainable = True
    set_trainable = False
    for layer in vgg.layers:
        if layer.name in ['block5_conv1', 'block4_conv1']:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    layers = [(layer, layer.name, layer.trainable) for layer in vgg.layers]
    pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])  

    t_x, t_y = next(train_generator)

    in_lay = Input(t_x.shape[1:])


    pt_features = vgg(in_lay)

    x = Flatten()(pt_features)
    dense1 = Dense(opt["Dense.0"], activation='relu')(x)
    dense2 = Dense(opt["Dense.1"], activation='relu')(dense1)
    prediction = Dense(num_class, activation='softmax')(dense2)
    
    lr = opt["LearningRate.0"]
    epochs = 50
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr), metrics=['accuracy','categorical_accuracy',f1_m,recall_m,precision_m,
                           SpecificityAtSensitivity(0.95), AUC(),cohen_quad])
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=opt["StoppingPatience.0"], verbose=1, mode='auto')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto',
                                  verbose=1)
    callbacks_list = [early_stop, reduce_lr]
    
    history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batchsize,
                    epochs=epochs,
                    validation_data = validation_generator, 
                    validation_steps = validation_generator.samples // batchsize,
                    verbose=1,
                    shuffle=True,
                    callbacks = callbacks_list
                    )
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']


    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    t = f.suptitle('VGG19_PSO_V1_test', fontsize=12)
    f.subplots_adjust(top=0.85, wspace=0.3)

    ax1.plot(acc, label='Train Accuracy')
    ax1.plot(val_acc, label='Validation Accuracy')
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    l1 = ax1.legend(loc="best")

    ax2.plot(loss, label='Train Loss')
    ax2.plot(val_loss, label='Validation Loss')
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    l2 = ax2.legend(loc="best")

    f.savefig(r'/notebooks/Saved Plots/Accuracy_Loss/VGG19/VGG19_PSO_V1_test')
    
    
    model.save('Saved Models/VGG19_PSO_V1_test.h5')   
    print('Saved Model to disk')
    
    from keras.preprocessing.image import ImageDataGenerator
    img_height = 224
    img_width = 224
    eval_data_dir = r'/notebooks/Pool2_Preprocessed_V2/test'
    eval_datagen = ImageDataGenerator(rescale=1./255) 

    eval_generator = eval_datagen.flow_from_directory(
        eval_data_dir,
        target_size=(img_height, img_width),
        batch_size=batchsize,
        class_mode='categorical',
        shuffle=True) 

    filenames = eval_generator.filenames
    nb_samples = len(filenames)
    
    predictions = model.predict(eval_generator,nb_samples)
    
    num_classes = 5
    import pandas as pd
    truth_array = tf.argmax(pd.get_dummies(pd.Series(eval_generator.classes)),axis=1)
    predictions_array = tf.argmax(predictions,axis=1)
    print(truth_array)
    print(predictions_array)
    
    from sklearn.metrics import accuracy_score, classification_report
    print('Accuracy on Test Data: %2.2f%%' % (accuracy_score(truth_array, predictions_array)))
    print(classification_report(truth_array,predictions_array))

    file1 = open(r'/notebooks/Saved Plots/Accuracy_TestData/VGG19/VGG19_PSO_V1_test.txt',"w")
    file1.write(str((accuracy_score(truth_array, predictions_array))))
    file1.write("\n")
    file1.write(str(classification_report(truth_array,predictions_array)))
    file1.close()
    
    print(max(history.history['val_cohen_kappa']))
    
    #compute conf mat
    cf_matrix = tf.math.confusion_matrix(truth_array,predictions_array)
    print(cf_matrix)

    import seaborn as sns
    fig, ax = plt.subplots()
    ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
                fmt='.2%', cmap='Blues')
    fig.savefig(r'/notebooks/Saved Plots/Confusion_Matrix/VGG19/VGG_PSO_V1')
    
    from sklearn.metrics import roc_curve, roc_auc_score
    sick_vec = truth_array>0
    sick_score = np.sum(predictions[:,1:],1)
    fpr, tpr, _ = roc_curve(sick_vec, sick_score)
    fig, ax1 = plt.subplots(1,1, figsize = (6, 6), dpi = 150)
    ax1.plot(fpr, tpr, 'b.-', label = 'Model Prediction (AUC: %2.2f)' % roc_auc_score(sick_vec, sick_score))
    ax1.plot(fpr, fpr, 'g-', label = 'Random Guessing')
    ax1.legend()
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')


    fig.savefig(r'/notebooks/Saved Plots/AUC_ROC/VGG19/VGG19_PSO_V1_test')
    
    
    return (1 - min(history.history['val_loss']))

In [ ]:
from hyperactive import Hyperactive, ParticleSwarmOptimizer, RandomSearchOptimizer
import hyperactive
search_space = {
    #"Dropout.0": list((0.2, 0.4, 0.6, 0.8)),
    #"Dense.0": list((32, 64, 128, 256, 512,1024)),
    #"Dense.1": list((32, 64, 128, 256, 512,1024)),
    #"LearningRate.0": list((0.00001, 0.0001, 0.001, 0.01, 0.1)),
    #"StoppingPatience.0": list((5,10,15,20,25))
    
    
    "Dense.0": list((128, 128)),
    "Dense.1": list((128, 128)),
    "LearningRate.0": list((0.001, 0.001)),
    "StoppingPatience.0": list((15, 15))
}

optimizer123 = ParticleSwarmOptimizer(
    inertia=0.4,
    cognitive_weight=0.7,
    social_weight=0.7,
    temp_weight=0.3,
    rand_rest_p=0.05,
)

hyper = Hyperactive()
hyper.add_search(cnn_for_pso, search_space, optimizer=optimizer123, n_iter=1)
hyper.run()